<a href="https://colab.research.google.com/github/YWANG1224/Colab/blob/main/smpdata_classificationwithNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Classification: finding optimal classifier

In [1]:
pip install transformers

     |████████████████████████████████| 3.4 MB 5.5 MB/s 
     |████████████████████████████████| 895 kB 38.5 MB/s 
     |████████████████████████████████| 61 kB 450 kB/s 
     |████████████████████████████████| 3.3 MB 32.2 MB/s 
     |████████████████████████████████| 596 kB 40.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
# pip install tensorflow

In [3]:
# pip install keras

In [4]:
import torch

In [5]:
import transformers

In [6]:
# Load packages
import matplotlib.pyplot as plt
import numpy as np
import jieba
import jieba.posseg as pseg
import re
import jieba.analyse as analyse
import gensim
import sklearn
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [7]:
# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [10]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User file "{name}"'.format(
      name=fn, length= len(uploaded[fn])
  ))

Saving ROI_training_yw.csv to ROI_training_yw.csv
User file "ROI_training_yw.csv"


In [11]:
# Load classification set
df_classification = pd.read_csv('ROI_training_yw.csv')

In [12]:
df = df_classification[['content','Pandemic (mainland related)']]


In [13]:
df = df.rename(columns={"content": "sentence", "Pandemic (mainland related)": "label"})

In [14]:
#df = df.set_index('_id')

In [15]:
# Report the number of sentences.
#df = df.drop([0])
df = df.dropna(axis=0, how='any')
df = df[df['label'].isin(["1","0"])]
print(df)
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

                                               sentence  label
0     【铁路部门公布进出北京地区列车免费退票措施】中国国家铁路集团有限公司宣布，自2020年6月1...      1
1     【直播！#全国性哀悼活动#】为表达全国各族人民对抗击新冠肺炎疫情斗争牺牲烈士和逝世同胞的深切...      1
2     【#李文亮医生有关情况调查的通报公布# 】国家监委调查组发布《关于群众反映的涉及李文亮医生有...      1
3     晚上和好朋友吃火锅，她说最近收了好多年轻的糖尿病病人…没有家族史。共同的都是疫情期间连续3个...      0
4     #如何劝人疫情期间别出门#？哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈！面对全球不断蔓延的疫情，...      0
...                                                 ...    ...
2995  【208年来首次，#新英格兰医学杂志称美国抗疫失败得令人震惊#】国际权威医学期刊《新英格兰医...      0
2996  知道新冠病毒为什么没有在美国全面爆发了吗？[doge]  世界神奇视频的微博视频 转发理由:...      0
2997  #大连理工大学# 打开微博看到了这样的热搜，很难过，刚刚经历过疫情期间研三论文的折磨，好多东...      1
2998  据央视财经，作为5G手机的核心部件，射频芯片市场大部分被美国和日本的几家公司占据，近期由于疫...      0
2999      『全国新型肺炎疫情实时动态 - 丁香园·丁香医生』http://t.cn/A6vBTRv3      1

[3000 rows x 2 columns]
Number of training sentences: 3,000



In [16]:
df.head(10)

,sentence,label
0,【铁路部门公布进出北京地区列车免费退票措施】中国国家铁路集团有限公司宣布，自2020年6月1...,1
1,【直播！#全国性哀悼活动#】为表达全国各族人民对抗击新冠肺炎疫情斗争牺牲烈士和逝世同胞的深切...,1
2,【#李文亮医生有关情况调查的通报公布# 】国家监委调查组发布《关于群众反映的涉及李文亮医生有...,1
3,晚上和好朋友吃火锅，她说最近收了好多年轻的糖尿病病人…没有家族史。共同的都是疫情期间连续3个...,0
4,#如何劝人疫情期间别出门#？哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈！面对全球不断蔓延的疫情，...,0
5,伊朗报告新增2566例新冠肺炎确诊病例，周四为2652例。,0
6,#武汉加油# #邓紫棋 平凡天使# 六天前在网上看到新闻，一位小伙子到派出所送上500个口罩...,1
7,身为小淘气的你或许已经发现了❗️@淘票票 官方微博粉丝正式突破200万大关👏未来依旧一起看电...,0
8,#杭州小区##杭州封小区##杭州所有村庄小区单位封闭式管理##对原有外地租客不能一概拒绝入住...,1
9,【#北京确诊送餐员活动范围公布#】该病例47岁，6月1日至17日每日通过饿了么平台接单送餐，...,1


In [17]:
sentences = df.sentence.values
labels = np.array(df.label.values, dtype=np.int32)

In [18]:
from transformers import BertTokenizer
from transformers import AutoTokenizer, AutoModelForMaskedLM
# Load the BERT tokenizer
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('hfl/chinese-bert-wwm-ext')

Loading BERT tokenizer...


Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

In [19]:
# Tokenize all of the sentences and map the tokens to their word IDs.
input_ids = []
# for every sentence...
for sent in sentences:
  # 'encode' will:
  # (1) tokenize the sentence.
  # (2) prepend the '[CLS]' token to the start.
  # (3) append the '[SEP]' token to the end.
  # (4) map tokens to their IDs.
  encoded_sent = tokenizer.encode(
      sent,  # sentence to encode.
      add_special_tokens = True, # add '[CLS]' and '[SEP]'
      # this function also supports truncation and conversion
      # to pytorch sensors, but we need to do padding, so we can't use features
      # max_length = 128,  # truncate all sentences.
      # return_tensors = 'pt', #return pytorch tensors.
  )
  # add the encoded sentence to the list.
  input_ids.append(encoded_sent)

# print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])
print('Max sentence length: ', max([len(sen) for sen in input_ids]))

Original:  【铁路部门公布进出北京地区列车免费退票措施】中国国家铁路集团有限公司宣布，自2020年6月17日0时起，旅客在车站、12306网站等各渠道，办理2020年6月16日24时前已购的进出北京地区各次列车有效车票退票时，均不收取退票手续费，购买铁路乘意险的一同办理。http://t.cn/A6LV2Iv4 转发理由:为铁路部门迅速行动点赞！一切为了战胜疫情。
Token IDs: [101, 523, 7188, 6662, 6956, 7305, 1062, 2357, 6822, 1139, 1266, 776, 1765, 1277, 1154, 6756, 1048, 6589, 6842, 4873, 2974, 3177, 524, 704, 1744, 1744, 2157, 7188, 6662, 7415, 1730, 3300, 7361, 1062, 1385, 2146, 2357, 8024, 5632, 8439, 2399, 127, 3299, 8126, 3189, 121, 3198, 6629, 8024, 3180, 2145, 1762, 6756, 4991, 510, 11914, 5381, 4991, 5023, 1392, 3940, 6887, 8024, 1215, 4415, 8439, 2399, 127, 3299, 8121, 3189, 8125, 3198, 1184, 2347, 6579, 4638, 6822, 1139, 1266, 776, 1765, 1277, 1392, 3613, 1154, 6756, 3300, 3126, 6756, 4873, 6842, 4873, 3198, 8024, 1772, 679, 3119, 1357, 6842, 4873, 2797, 5330, 6589, 8024, 6579, 743, 7188, 6662, 733, 2692, 7372, 4638, 671, 1398, 1215, 4415, 511, 8184, 131, 120, 120, 162, 119, 8274, 120, 11716, 9084, 8144, 8169, 12507, 6760, 1355, 4415, 4507, 131, 71

In [20]:
# check the sentence length (need to have a unified length of the sentence for further processing)
len(input_ids)
s = 0
for i in input_ids:
  if len(i) > 140:
    s +=1
print(s)

1819


In [21]:
from keras.preprocessing.sequence import pad_sequences
# set the max sequence length, choose *** to drop the extreme values
MAX_LEN = 140
print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)
print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))
# pad our input tokens with value 0.
# "post" indicates that we want to pad and truncate at the end of the sequence as opposed to the beginning.
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype='long',
                          value=0, truncating="post", padding="post")
print('\Done.')


Padding/truncating all sentences to 140 values...

Padding token: "[PAD]", ID: 0
\Done.


In [22]:
# creating attention masks
attention_masks = []
# for each sentence...
for sent in input_ids:
  # create the attention mask
  # - if a token id is 0, then it's padding, set the mask to 0
  # - if a token id is >0, then it's a real token, set the mask to 1
  att_mask = [int(token_id >0) for token_id in sent]

  # store the attention mask for this sentence
  attention_masks.append(att_mask)

In [23]:
# use train_test_split to split the data into train and validation sets for training
from sklearn.model_selection import train_test_split
# use 90% for training and 10% for validation
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                                                    random_state=2022, 
                                                                                    test_size=0.1)
# do the same for the masks
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels, 
                                                       random_state=2022, test_size=0.1)

In [24]:
# convert all inputs and labels to torch tensors, the required datatype for the model
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [25]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# the DataLoader needs to know the batch size for training, so need to specify it here.
# for fine-tuning BERT on a specific task, a recommended batch size is between 16 to 32.

batch_size = 16 # the larger the batch_size, the larger cpu/gpu reqiured

# create the DataLoader for the training set.
print(train_inputs.size(), train_masks.size(), train_labels.size())
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# create the DataLoader for the validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

torch.Size([2700, 140]) torch.Size([2700, 140]) torch.Size([2700])


In [26]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "hfl/chinese-bert-wwm-ext",
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
#     "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
#     num_labels = 2, # The number of output labels--2 for binary classification.
#                     # You can increase this for multi-class tasks.   
#     output_attentions = False, # Whether the model returns attentions weights.
#     output_hidden_states = False, # Whether the model returns all hidden-states.
# )
# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [27]:
# get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())
print('The BERT model has {:} different named parameters.\n'.format(len(params)))
print('====Embedding Layer ====\n')
for p in params[0:5]:
  print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== First Transformer ====\n')
for p in params[5:21]:
  print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== Output Layer ====\n')
for p in params[-4:]:
  print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

====Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (21128, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (7

In [28]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
from transformers import get_linear_schedule_with_warmup
# Number of training epochs (authors recommend between 2 and 4)
epochs = 1
# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs
print(total_steps)
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

169


In [29]:
import numpy as np
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
import time
import datetime
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [30]:
import random
# Set the seed value all over the place to make this reproducible.
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
# Store the average loss after each epoch so we can plot them.
loss_values = []
# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    # Measure how long the training epoch takes.
    t0 = time.time()
    # Reset the total loss for this epoch.
    total_loss = 0
    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()
    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].long().to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].long().to(device)
        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        
        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()
        # Update the learning rate.
        scheduler.step()
    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.
    print("")
    print("Running Validation...")
    t0 = time.time()
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()
    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy
        # Track the number of batches
        nb_eval_steps += 1
    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
print("")
print("Training complete!")


======== Epoch 1 / 1 ========
Training...
  Batch    40  of    169.    Elapsed: 0:00:34.
  Batch    80  of    169.    Elapsed: 0:01:08.
  Batch   120  of    169.    Elapsed: 0:01:42.
  Batch   160  of    169.    Elapsed: 0:02:16.

  Average training loss: 0.39
  Training epcoh took: 0:02:23

Running Validation...
  Accuracy: 0.89
  Validation took: 0:00:06

Training complete!


In [31]:
import pandas as pd
import plotly.express as px
f = pd.DataFrame(loss_values)
f.columns=['Loss']
fig = px.line(f, x=f.index, y=f.Loss)
fig.update_layout(title='Training loss of the Model',
                   xaxis_title='Epoch',
                   yaxis_title='Loss')
fig.show()


In [32]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User file "{name}"'.format(
      name=fn, length= len(uploaded[fn])
  ))

Saving ROI_test.csv to ROI_test.csv
User file "ROI_test.csv"


In [34]:
import pandas as pd
# Load the dataset into a pandas dataframe.
df_test = pd.read_csv("ROI_test.csv")
df = df_test[['content']]
df = df.rename(columns={"content": "sentence"})
df["label"] = ""
# Report the number of sentences.
print(df)
print('Number of training sentences: {:,}\n'.format(df.shape[0]))
print('Number of test sentences: {:,}\n'.format(df.shape[0]))

                                              sentence label
0    【#特朗普回应权贵优先接受核酸检测#：这就是人生吧，这种情况有时发生】当地时间3月18日，《...      
1    我分享了【2020年3月13日0时至24时辽宁新型冠状病毒肺炎疫情情况】, 快来看吧！ | ...      
2    #如何劝人疫情期间别出门#？哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈！面对全球不断蔓延的疫情，...      
3    关于松本梨香所谓的【冒领新冠辅助金】那件事，我也看了，给大家捋一下，疫情期间东京都那边搞了个...      
4    【#江西新增确诊85例#，#江西累计确诊476例#】2020年2月3日0-24时，江西省报告...      
..                                                 ...   ...
995  🌈花姐温馨提示： 1⃣️增强卫生健康意识，注意防寒保暖、室内通风和个人卫生  2⃣️加强体育...      
996  五一出不去[泪]在家云旅游[doge]，大家不如来会呼吸的江湖瞅瞅，足不出户就可体验#逆水寒...      
997  这次疫情全国上下为了那么几个傻缺买单，受连累的何止现在，何止经济。还有患难见真情。MDZZ，...      
998  #BTOB[超话]#  [公告]BTOB 4U COVID-19检查结果及行程相关通知 大家...      
999  互联网是有记忆的 转发理由://@风息神泪:我前两天找物资的时候，朋友联系的其中一家，是疫情...      

[1000 rows x 2 columns]
Number of training sentences: 1,000

Number of test sentences: 1,000



In [36]:
# Create sentence and label lists
sentences = df.sentence.values
# labels = np.array(df.label.values,dtype=np.int32)
labels = [[] for _ in range(len(df))] 
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
# For every sentence...
for sent in sentences:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                   )
    
    input_ids.append(encoded_sent)
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, 
                          dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 
# Convert to tensors.
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
# Set the batch size.  
batch_size = 32  
# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)


In [40]:
# Prediction on test set
print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))
# Put model in evaluation mode
model.eval()
# Tracking variables 
predictions , true_labels = [], []
# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)
  logits = outputs[0]
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)
print('DONE.')
#print('Positive samples: %d of %d (%.2f%%)' % (df.label.sum(), len(df.label), (df.label.sum() / len(df.label) * 100.0)))


Predicting labels for 1,000 test sentences...
DONE.


In [42]:
predictions

[array([[ 2.0197287 , -2.0835128 ],
        [-1.246795  ,  1.4482628 ],
        [ 1.8161411 , -2.1489606 ],
        [ 1.7741764 , -1.7878028 ],
        [-1.0819647 ,  1.3603362 ],
        [ 2.0604281 , -2.0702763 ],
        [-1.698974  ,  1.7548178 ],
        [ 1.8700846 , -1.9385251 ],
        [ 1.1230977 , -0.9039299 ],
        [ 2.005951  , -1.9422914 ],
        [-1.8183019 ,  1.6552438 ],
        [ 1.9213035 , -1.9198005 ],
        [-1.0327786 ,  1.19282   ],
        [ 1.8808521 , -1.8582917 ],
        [ 1.3483982 , -1.5572258 ],
        [ 1.7483586 , -1.4146882 ],
        [-1.9108354 ,  1.8077574 ],
        [ 1.9316032 , -1.7176708 ],
        [-2.0928655 ,  1.6170576 ],
        [ 1.5835586 , -1.7530048 ],
        [ 1.5672423 , -1.6904254 ],
        [ 1.5113792 , -1.4105558 ],
        [ 0.32748505, -0.41237757],
        [-1.4290651 ,  1.5873392 ],
        [ 1.6416082 , -1.768826  ],
        [-1.505875  ,  1.4695796 ],
        [-0.8975384 ,  1.1239958 ],
        [ 1.3713015 , -1.462

In [43]:
from sklearn.metrics import matthews_corrcoef
#matthews_set = []
# Evaluate each test batch using Matthew's correlation coefficient
#print('Calculating Matthews Corr. Coef. for each batch...')
# For each input batch...
for i in range(len(true_labels)):
  
  # The predictions for this batch are a 2-column ndarray (one column for "0" 
  # and one column for "1"). Pick the label with the highest value and turn this
  # in to a list of 0s and 1s.
  pred_labels_i = np.argmax(predictions[i], axis=1).flatten()
  
  # Calculate and store the coef for this batch.  
  #matthews = matthews_corrcoef(true_labels[i], pred_labels_i)                
  #matthews_set.append(matthews)
  # Combine the predictions for each batch into a single list of 0s and 1s.
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
# Combine the correct labels for each batch into a single list.
#flat_true_labels = [item for sublist in true_labels for item in sublist]
# Calculate the MCC
#mcc = matthews_corrcoef(flat_true_labels, flat_predictions)
#print('MCC: %.3f' % mcc)


In [45]:
len(flat_predictions)

1000

In [49]:
df['label_pred'] = flat_predictions.tolist()

In [50]:
df.head(10)

,sentence,label,label_pred
0,【#特朗普回应权贵优先接受核酸检测#：这就是人生吧，这种情况有时发生】当地时间3月18日，《...,,0
1,"我分享了【2020年3月13日0时至24时辽宁新型冠状病毒肺炎疫情情况】, 快来看吧！ | ...",,1
2,#如何劝人疫情期间别出门#？哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈！面对全球不断蔓延的疫情，...,,0
3,关于松本梨香所谓的【冒领新冠辅助金】那件事，我也看了，给大家捋一下，疫情期间东京都那边搞了个...,,0
4,【#江西新增确诊85例#，#江西累计确诊476例#】2020年2月3日0-24时，江西省报告...,,1
5,【#特朗普弄错首例新冠肺炎逝者性别#：“她是了不起的女性”[衰]】当地时间2月29日，美国总...,,0
6,#武汉一社区用环卫车运送平价肉# 看到网上该社区居民和社区负责人理论的视频，发现很多网友说...,,1
7,【#意大利宣布进入国家紧急状态# 将持续六个月】据安莎社报道，意大利总理孔特31日宣布，该国...,,0
8,我兑现了承诺！疫情期间在家不乱跑，还用七天时间瘦了十斤！ 荒原moor的微博视频 转发理由...,,0
9,【美国1月11日已启动疫苗研发 就在中国分享新冠病毒基因几个小时内】总统特朗普15日在白宫一...,,0


In [ ]:
#count = 0
#for i in range(len(flat_true_labels)):
#  if int(flat_predictions[i]) ==  int(flat_true_labels[i]):
#      count +=1
#print(count, "正确率： " count/len(flat_true_labels))
